# Run Stored Procedures 

Operationalize snowpark stored procedures

1. Use Snowpark Python Stored Procedures to run workloads

2. Create DAG executing stored procedures 

For more information follow the below links:

1.[Training Machine Learning Models with Snowpark Python](https://docs.snowflake.com/en/developer-guide/snowpark/python/python-snowpark-training-ml)

2.[Managing Snowflake tasks and task graphs with Python](https://docs.snowflake.com/en/developer-guide/snowflake-python-api/snowflake-python-managing-tasks)

In [ ]:
from snowflake.core import Root
from snowflake.snowpark.context import get_active_session

session = get_active_session()
root = Root(session)

In [ ]:
import time
from datetime import timedelta
from snowflake.core.task import StoredProcedureCall
from snowflake.core.task.dagv1 import DAG, DAGTask, DAGOperation
from snowflake.snowpark import Session

def wait1(session: Session) -> None:
    time.sleep(1)

with DAG("dag", schedule=timedelta(days=1)) as dag:
    task1 = DAGTask("task1", "select system$wait(2)")
    task2 = DAGTask("task2",
        StoredProcedureCall(wait1,
            stage_location="@stage1",
            packages=["snowflake-snowpark-python"]),
        warehouse="compute_wh")
    task3 = DAGTask("task3", "select system$wait(2)")
    task4 = DAGTask("task4",
        StoredProcedureCall(wait1,
            stage_location="@stage1",
            packages=["snowflake-snowpark-python"]),
        warehouse="compute_wh")
    
task1 >> task2 >> task4
task1 >> task3 >> task4
schema = root.databases["snowpark_db"].schemas["sourced"]
dagop = DAGOperation(schema)
dagop.deploy(dag)

In [ ]:
dagop.run("dag")


In [ ]:
dagop.get_current_dag_runs("dag")[0]

In [ ]:
dagop.get_complete_dag_runs("dag")[1]